# Домашнее задание № 7

### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [77]:
import nltk
import pandas as pd
nltk.download('wordnet')
import pymorphy2
import warnings
warnings.filterwarnings('ignore')

from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nanosemantics/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [78]:
def lesk(word, sentence):
    """Ваш код тут"""
    bestsense = 0
    maxoverlap = 0
    synsets = wn.synsets(word)
    
    for i, syns in enumerate(synsets):
        syns_definition = syns.definition().split()
        lemm_syns_definition = [lemmatizer.lemmatize(token) for token in syns_definition]
        cross_tokens = set(syns_definition) & set(sentence)
        if len(cross_tokens) > maxoverlap:
            maxoverlap = len(cross_tokens)
            bestsense = i
    return bestsense

Работать функция должна как-то так:

In [79]:
# на вход подается элемент результата работы уже написанной вами функции get_words_in_context
lesk('day', 'some point or period in time'.split()) # для примера контекст совпадает с одним из определений
# а на выходе индекс подходящего синсета

1

In [80]:
wn.synsets('day')[0].definition()

'time for Earth to make a complete rotation on its axis'

In [81]:
wn.synsets('day')[1].definition()

'some point or period in time'

**Проверьте насколько хорошо работает такой метод на датасете из семинара**

In [33]:
corpus_wsd = []
corpus = open('../compling_nlp_hse_course/data/corpus_wsd_50k.txt/corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

In [37]:
corpus_wsd[0]

[['', 'how', 'How'],
 ['long%3:00:02::', 'long', 'long'],
 ['', 'have', 'has'],
 ['', 'it', 'it'],
 ['be%2:42:03::', 'be', 'been'],
 ['', 'since', 'since'],
 ['', 'you', 'you'],
 ['review%2:31:00::', 'review', 'reviewed'],
 ['', 'the', 'the'],
 ['objective%1:09:00::', 'objective', 'objectives'],
 ['', 'of', 'of'],
 ['', 'you', 'your'],
 ['benefit%1:21:00::', 'benefit', 'benefit'],
 ['', 'and', 'and'],
 ['service%1:04:07::', 'service', 'service'],
 ['program%1:09:01::', 'program', 'program'],
 ['', '?', '?']]

Вот так можно достать соответствующий этому тэгу синсет:

In [46]:
wn.lemma_from_key('review%2:31:00::').synset()

Synset('review.v.01')

Когда с помощью алгоритма Леска вы найдете подходящее значение, их можно просто сравнить

In [47]:
wn.synsets('long')[0]

Synset('hanker.v.01')

In [48]:
wn.synsets('long')[1]

Synset('long.a.01')

In [49]:
lesk('long', wn.synsets('long')[0].definition())

0

In [50]:
lesk('long', wn.synsets('long')[1].definition())

1

In [65]:
# тут подходит второй синсет (!в вашем решении вместо индекса единицы будет результат работы функции lesk)
wn.synsets('long')[lesk('long', wn.synsets('long')[1].definition())] == wn.lemma_from_key('long%3:00:02::').synset()

True

In [66]:
wn.lemma_from_key('long%3:00:02::').synset()

Synset('long.a.01')

In [67]:
wn.lemma_from_key('long%3:00:02::').synset().definition()

'primarily temporal sense; being or indicating a relatively great or greater than average duration or passage of time or a duration as specified'

**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

In [135]:
tp = 0
fp = 0

for row in tqdm(corpus_wsd):
    if len(row) > 1:
        sentence_tokens = [token_info[1] for token_info in row]
        for token_info in row:
            if token_info[0]:
                token = token_info[1]
                best_ind = lesk(token, sentence_tokens)
                best_synset = wn.synsets(token)[best_ind]
                if best_synset == wn.lemma_from_key(token_info[0]).synset():
                    tp += 1
                else:
                    fp += 1




  0%|                                                                                        | 0/49453 [00:00<?, ?it/s]


  0%|                                                                              | 14/49453 [00:00<06:24, 128.43it/s]


  0%|                                                                              | 29/49453 [00:00<06:13, 132.44it/s]


  0%|                                                                              | 49/49453 [00:00<05:35, 147.05it/s]


  0%|                                                                              | 63/49453 [00:00<05:45, 143.08it/s]


  0%|                                                                              | 77/49453 [00:00<05:49, 141.28it/s]


  0%|▏                                                                             | 90/49453 [00:00<06:03, 135.96it/s]


  0%|▏                                                                            | 103/49453 [00:00<06:13, 132.07it/s]


  0%|▏               

  4%|██▉                                                                         | 1906/49453 [00:14<05:33, 142.51it/s]


  4%|██▉                                                                         | 1930/49453 [00:14<04:53, 161.96it/s]


  4%|██▉                                                                         | 1950/49453 [00:14<04:46, 165.77it/s]


  4%|███                                                                         | 1969/49453 [00:15<04:35, 172.36it/s]


  4%|███                                                                         | 2001/49453 [00:15<03:58, 198.93it/s]


  4%|███                                                                         | 2024/49453 [00:15<04:28, 176.74it/s]


  4%|███▏                                                                        | 2044/49453 [00:15<04:27, 177.28it/s]


  4%|███▏                                                                        | 2067/49453 [00:15<04:10, 189.43it/s]


  4%|███▏               

  8%|██████▏                                                                      | 4004/49453 [00:29<09:16, 81.73it/s]


  8%|██████▏                                                                      | 4014/49453 [00:30<08:52, 85.36it/s]


  8%|██████▎                                                                      | 4028/49453 [00:30<07:50, 96.48it/s]


  8%|██████▏                                                                     | 4041/49453 [00:30<07:21, 102.83it/s]


  8%|██████▏                                                                     | 4053/49453 [00:30<07:15, 104.35it/s]


  8%|██████▎                                                                     | 4067/49453 [00:30<06:43, 112.44it/s]


  8%|██████▎                                                                     | 4080/49453 [00:30<06:30, 116.24it/s]


  8%|██████▎                                                                     | 4093/49453 [00:30<06:29, 116.50it/s]


  8%|██████▎            

 12%|█████████                                                                    | 5802/49453 [00:45<07:50, 92.70it/s]


 12%|█████████                                                                    | 5813/49453 [00:45<07:52, 92.39it/s]


 12%|█████████                                                                    | 5823/49453 [00:45<08:23, 86.68it/s]


 12%|█████████                                                                    | 5836/49453 [00:45<07:42, 94.21it/s]


 12%|█████████                                                                    | 5846/49453 [00:45<07:34, 95.87it/s]


 12%|█████████                                                                   | 5858/49453 [00:45<07:07, 102.03it/s]


 12%|█████████▏                                                                   | 5869/49453 [00:45<07:25, 97.89it/s]


 12%|█████████▏                                                                   | 5880/49453 [00:45<07:17, 99.58it/s]


 12%|█████████          

 16%|███████████▊                                                                | 7666/49453 [00:59<04:46, 145.70it/s]


 16%|███████████▊                                                                | 7684/49453 [00:59<04:47, 145.18it/s]


 16%|███████████▊                                                                | 7701/49453 [00:59<05:20, 130.20it/s]


 16%|███████████▊                                                                | 7716/49453 [01:00<05:57, 116.60it/s]


 16%|███████████▉                                                                | 7730/49453 [01:00<05:53, 118.09it/s]


 16%|████████████                                                                 | 7743/49453 [01:00<07:09, 97.19it/s]


 16%|████████████                                                                 | 7755/49453 [01:00<07:14, 95.90it/s]


 16%|████████████                                                                 | 7766/49453 [01:00<07:24, 93.86it/s]


 16%|████████████       

...

 39%|█████████████████████████████▎                                             | 19308/49453 [02:29<04:19, 116.27it/s]


 39%|█████████████████████████████▎                                             | 19323/49453 [02:29<04:02, 124.37it/s]


 39%|█████████████████████████████▎                                             | 19339/49453 [02:29<03:47, 132.61it/s]


 39%|█████████████████████████████▎                                             | 19358/49453 [02:29<03:33, 140.95it/s]


 39%|█████████████████████████████▍                                             | 19373/49453 [02:29<03:42, 135.38it/s]


 39%|█████████████████████████████▍                                             | 19387/49453 [02:29<03:46, 132.84it/s]


 39%|█████████████████████████████▍                                             | 19412/49453 [02:29<03:15, 153.71it/s]


 39%|█████████████████████████████▍                                             | 19429/49453 [02:29<03:21, 148.71it/s]


 39%|███████████████████

 49%|████████████████████████████████████▊                                      | 24272/49453 [02:42<01:10, 358.42it/s]


 49%|████████████████████████████████████▊                                      | 24309/49453 [02:43<01:09, 359.70it/s]


 49%|████████████████████████████████████▉                                      | 24351/49453 [02:43<01:06, 374.87it/s]


 49%|████████████████████████████████████▉                                      | 24390/49453 [02:43<01:09, 359.36it/s]


 49%|█████████████████████████████████████                                      | 24429/49453 [02:43<01:08, 365.95it/s]


 50%|█████████████████████████████████████▏                                     | 24487/49453 [02:43<01:00, 411.51it/s]


 50%|█████████████████████████████████████▏                                     | 24531/49453 [02:43<00:59, 419.65it/s]


 50%|█████████████████████████████████████▎                                     | 24575/49453 [02:43<01:01, 405.54it/s]


 50%|███████████████████

 60%|█████████████████████████████████████████████▎                             | 29893/49453 [02:56<00:48, 399.60it/s]


 61%|█████████████████████████████████████████████▍                             | 29934/49453 [02:56<00:51, 382.37it/s]


 61%|█████████████████████████████████████████████▍                             | 29974/49453 [02:56<00:50, 385.25it/s]


 61%|█████████████████████████████████████████████▌                             | 30027/49453 [02:57<00:46, 419.62it/s]


 61%|█████████████████████████████████████████████▌                             | 30076/49453 [02:57<00:44, 438.51it/s]


 61%|█████████████████████████████████████████████▋                             | 30142/49453 [02:57<00:39, 485.44it/s]


 61%|█████████████████████████████████████████████▊                             | 30196/49453 [02:57<00:38, 500.60it/s]


 61%|█████████████████████████████████████████████▊                             | 30248/49453 [02:57<00:39, 482.31it/s]


 61%|███████████████████

 72%|█████████████████████████████████████████████████████▊                     | 35500/49453 [03:10<00:31, 442.64it/s]


 72%|█████████████████████████████████████████████████████▉                     | 35546/49453 [03:10<00:32, 421.83it/s]


 72%|█████████████████████████████████████████████████████▉                     | 35589/49453 [03:10<00:33, 413.23it/s]


 72%|██████████████████████████████████████████████████████                     | 35631/49453 [03:10<00:34, 404.43it/s]


 72%|██████████████████████████████████████████████████████                     | 35674/49453 [03:10<00:33, 411.77it/s]


 72%|██████████████████████████████████████████████████████▏                    | 35716/49453 [03:10<00:36, 381.47it/s]


 72%|██████████████████████████████████████████████████████▏                    | 35755/49453 [03:11<00:36, 375.11it/s]


 72%|██████████████████████████████████████████████████████▎                    | 35802/49453 [03:11<00:34, 399.29it/s]


 72%|███████████████████

 92%|█████████████████████████████████████████████████████████████████████      | 45520/49453 [03:23<00:04, 848.50it/s]


 92%|█████████████████████████████████████████████████████████████████████▏     | 45610/49453 [03:23<00:04, 863.30it/s]


 92%|█████████████████████████████████████████████████████████████████████▎     | 45717/49453 [03:23<00:04, 914.05it/s]


 93%|█████████████████████████████████████████████████████████████████████▍     | 45810/49453 [03:23<00:04, 889.74it/s]


 93%|█████████████████████████████████████████████████████████████████████▌     | 45906/49453 [03:23<00:03, 909.70it/s]


 93%|█████████████████████████████████████████████████████████████████████▊     | 46000/49453 [03:24<00:03, 918.56it/s]


 93%|█████████████████████████████████████████████████████████████████████▉     | 46093/49453 [03:24<00:03, 862.91it/s]


 93%|██████████████████████████████████████████████████████████████████████     | 46181/49453 [03:24<00:03, 867.95it/s]


 94%|███████████████████

In [137]:
print('Accuracy: ', round(tp / (tp + fp), 4))

Accuracy:  0.3401


### Задание 2* (2 балла)

В семинаре для WSI на данных Диалога использовался только Fastext. Попробуйте заменить его на адаграм (обучите свою модель или используйте предобученную out.pkl или https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib), а также поэкспериментируйте с разными алгоритмами кластеризации и их параметрами (можно использовать только те алгоритмы, которые обучаются достаточно быстро)

Для каждого эксперимента рассчитайте ARI

In [39]:
import adagram
import numpy as np
from nltk.corpus import stopwords
from string import punctuation
from pymorphy2 import MorphAnalyzer
from razdel import tokenize as razdel_tokenize
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import *


morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [13]:
df = pd.read_csv('train.txt', sep='\t')

In [8]:
!wget 'https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib'
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

--2022-02-07 18:41:15--  https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.40.98
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.40.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1462416741 (1,4G) [application/x-www-form-urlencoded]
Saving to: ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib’

all.a010.p10.d300.w 100%[===================>]   1,36G  14,0MB/s    in 2m 26s  

2022-02-07 18:43:42 (9,58 MB/s) - ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib’ saved [1462416741/1462416741]



In [29]:
def get_embedding_adagram(text, model, window, dim):
    
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))
    
    vectors = np.zeros((len(word2context), dim))
    
    for i,word in enumerate(word2context):
        word, context = word
        try:
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v 

        except (KeyError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [30]:
def normalize(text):
    
    words = [token.text.strip(punct) for token in list(razdel_tokenize(text))]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [31]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [56]:
df['gold_sense_id'].unique()

array([1, 2])

In [47]:
for key, _ in grouped_df:
    print(key)

бор
замок
лук
суда


In [65]:
window = 6
dim = 300

ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].apply(normalize)
    X = np.zeros((len(texts), dim))
    
    for i, text in enumerate(texts):
        text = [word for word in text if word != key]
        X[i] = get_embedding_adagram(text, vm, window, dim)
        
    cluster = KMeans(2, n_init=20, max_iter=500)
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1
    
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print('ARI: ', np.mean(ARI))

ARI:  0.5013583891990683


In [69]:
window = 6
dim = 300

ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].apply(normalize)
    X = np.zeros((len(texts), dim))
    
    for i, text in enumerate(texts):
        text = [word for word in text if word != key]
        X[i] = get_embedding_adagram(text, vm, window, dim)
        
    cluster = AffinityPropagation(damping=0.5)
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1
    
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print('ARI:', np.mean(ARI))

ARI: 0.09408947987940999


In [76]:
window = 6
dim = 300

ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].apply(normalize)
    X = np.zeros((len(texts), dim))
    
    for i, text in enumerate(texts):
        text = [word for word in text if word != key]
        X[i] = get_embedding_adagram(text, vm, window, dim)
        
    cluster = AgglomerativeClustering(n_clusters=2)
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1
    
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print('ARI:', np.mean(ARI))

ARI: 0.634245244648562


Мы немного сжульничали, с учетом того, что знали, что кластера в датасете всего 2. Но в любом случае аггломеративная кластеризация показала наилучший результат, ARI - ближе всех к 1. Хотя и обычный kmeans тоже неплох.